In [ ]:
import os
import numpy as np
import pandas as pd
from random import gauss

padding_value = 0

# Function to read a CSV file as a binary matrix
def read_csv_as_binary_matrix(file_path):
    df = pd.read_csv(file_path, header=None, delimiter=";", dtype=int)
    return df.values

# Define functions for calculating aggregated matrices
def calculate_aggregated_matrix_2x2(original_matrix):
    num_rows, num_cols = original_matrix.shape
    aggregated_matrix = np.zeros((num_rows // 2, num_cols // 2), dtype=float)

    for i in range(0, num_rows, 2):
        for j in range(0, num_cols, 2):
            square = original_matrix[i:i + 2, j:j + 2]
            average = square.mean()
            row_idx = i // 2
            col_idx = j // 2
            aggregated_matrix[row_idx, col_idx] = round(average, 2)

    return aggregated_matrix

def calculate_aggregated_matrix_3x3(original_matrix):
    num_rows, num_cols = original_matrix.shape
    aggregated_matrix = np.zeros((num_rows // 3, num_cols // 3), dtype=float)

    for i in range(0, num_rows, 3):
        for j in range(0, num_cols, 3):
            square = original_matrix[i:i + 3, j:j + 3]
            average = square.mean()
            row_idx = i // 3
            col_idx = j // 3
            aggregated_matrix[row_idx, col_idx] = round(average, 2)

    return aggregated_matrix

def calculate_aggregated_matrix_4x4(original_matrix):
    num_rows, num_cols = original_matrix.shape
    aggregated_matrix = np.zeros((num_rows // 4, num_cols // 4), dtype=float)

    for i in range(0, num_rows, 4):
        for j in range(0, num_cols, 4):
            square = original_matrix[i:i + 4, j:j + 4]
            average = square.mean()
            row_idx = i // 4
            col_idx = j // 4
            aggregated_matrix[row_idx, col_idx] = round(average, 2)

    return aggregated_matrix

# NM Landscape

def calculate_betas(aggregated_matrix, Sigma, M):
    element_num = aggregated_matrix.shape[0]
    interaction_coeffi1 = np.zeros((element_num, element_num))
    interaction_coeffi2 = np.zeros((element_num, element_num))

    for i in range(element_num):
        interaction_coeffi1[0, i] = np.exp(-abs(gauss(aggregated_matrix[i, i], Sigma)))
        interaction_genes = np.random.choice(np.delete(np.arange(element_num), i), M, replace=False)
        for j in interaction_genes:
            interaction_coeffi2[i, j] = np.exp(-abs(gauss(aggregated_matrix[i, j], Sigma)))

    return interaction_coeffi1, interaction_coeffi2

def calculate_fitness(X, interaction_coeffi1, interaction_coeffi2, aggregated_matrix, M):
    element_num = aggregated_matrix.shape[0]
    fitness = 0.0

    for i in range(element_num):
        fitness += interaction_coeffi1[0, i] * X[i]
        interaction_genes = np.random.choice(np.delete(np.arange(element_num), i), M, replace=False)
        for j in interaction_genes:
            fitness += interaction_coeffi2[i, j] * X[i] * X[j]

    return fitness

def generate_landscape(n, aggregated_matrix, interaction_coeffi1, interaction_coeffi2, M):
    fitness_vec = []
    fitness_pos = []

    for rr in range(pow(2, n)):
        binary_vector = format(rr, f'0{n}b')
        position_11 = int(binary_vector, 2)
        fitness_pos.append(position_11)

        binary_vector = ','.join(binary_vector)
        binary_vector = binary_vector.replace('0', '-1')

        X = [int(x) for x in binary_vector.split(",")]

        fitness = calculate_fitness(X, interaction_coeffi1, interaction_coeffi2, aggregated_matrix, M)
        fitness_vec.append(fitness)

    return fitness_vec, fitness_pos

def local_optima(fitness_vector, fitness_pos, Sigma, landscape_id, file_name):
    local_optima_results = []
    count_local_optima = 0

    for i in range(len(fitness_vector)):
        binary_vector = format(fitness_pos[i], f'0{n}b')
        binary_vector = binary_vector.replace('-1', '0')
        position_11 = int(binary_vector, 2)
        index_position_11 = fitness_pos.index(position_11)
        fitness_local = fitness_vector[index_position_11]
        is_local_optima = True

        for j in range(len(binary_vector)):
            neighbor_binary_vector_11 = list(binary_vector)

            if neighbor_binary_vector_11[j - 1] == '0':
                neighbor_binary_vector_11[j - 1] = '1'
            else:
                neighbor_binary_vector_11[j - 1] = '0'

